In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import numpy as np

import torch
import torch.nn.functional as F
from ctpn_model import CTPN_Model
from ctpn_utils import gen_anchor, bbox_transfor_inv, clip_box, filter_bbox,nms, TextProposalConnectorOriented
from ctpn_utils import resize
import config

import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
prob_thresh = 0.5
width = 960
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# weights = '/remote-home/my/final/Code/OCR-code(CTPN+CRNN+CTC Loss)-baidu/checkpoints/v3_ctpn_ep22_0.4395_0.1096_0.5491.pth'#'ctpn_ep17_0.0544_0.1125_0.1669.pth')
# weights = '/remote-home/my/final/Code/OCR-code(CTPN+CRNN+CTC Loss)-baidu/CTPN.pth'

# model = CTPN_Model()
# model.load_state_dict(torch.load(weights, map_location=device)['model_state_dict'])
# model.to(device)
# model.eval()


def dis(image):
    cv2.imshow('image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


def get_det_boxes(model, image,display = True):
#     image = resize(image, height=720)
    image_c = image.copy()
    h, w = image.shape[:2]
    image = image.astype(np.float32) - config.IMAGE_MEAN
    image = torch.from_numpy(image.transpose(2, 0, 1)).unsqueeze(0).float()

    with torch.no_grad():
        image = image.to(device)
        cls, regr = model(image)
        cls_prob = F.softmax(cls, dim=-1).cpu().numpy()
        regr = regr.cpu().numpy()
        anchor = gen_anchor((int(h / 16), int(w / 16)), 16)
        bbox = bbox_transfor_inv(anchor, regr)
        bbox = clip_box(bbox, [h, w])
        # print(bbox.shape)

        fg = np.where(cls_prob[0, :, 1] > prob_thresh)[0]
        # print(np.max(cls_prob[0, :, 1]))
        select_anchor = bbox[fg, :]
        select_score = cls_prob[0, fg, 1]
        select_anchor = select_anchor.astype(np.int32)
        # print(select_anchor.shape)
        keep_index = filter_bbox(select_anchor, 16)

        # nms
        select_anchor = select_anchor[keep_index]
        select_score = select_score[keep_index]
        select_score = np.reshape(select_score, (select_score.shape[0], 1))
        nmsbox = np.hstack((select_anchor, select_score))
        keep = nms(nmsbox, 0.3)
        # print(keep)
        select_anchor = select_anchor[keep]
        select_score = select_score[keep]

        # text line-
        textConn = TextProposalConnectorOriented()
        text = textConn.get_text_lines(select_anchor, select_score, [h, w])
#         print(text)
        if display:
            for i in text:
                s = str(round(i[-1] * 100, 2)) + '%'
                i = [int(j) for j in i]
                cv2.line(image_c, (i[0], i[1]), (i[2], i[3]), (0, 0, 255), 2)
                cv2.line(image_c, (i[0], i[1]), (i[4], i[5]), (0, 0, 255), 2)
                cv2.line(image_c, (i[6], i[7]), (i[2], i[3]), (0, 0, 255), 2)
                cv2.line(image_c, (i[4], i[5]), (i[6], i[7]), (0, 0, 255), 2)
                cv2.putText(image_c, s, (i[0]+13, i[1]+13),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (255,0,0),
                            2,
                            cv2.LINE_AA)
                

        return text,image_c

In [3]:
# 计算IOU

import numpy as np 
import shapely
from shapely.geometry import Polygon,MultiPoint  #多边形
 
def cal_iou(line1,line2):
    # line1=[908,215,934,312,752,355,728,252]   #四边形四个点坐标的一维数组表示，[x,y,x,y....]
    a=np.array(line1).reshape(4, 2)   #四边形二维坐标表示
    poly1 = Polygon(a).convex_hull  #python四边形对象，会自动计算四个点，最后四个点顺序为：左上 左下  右下 右上 左上

    # line2=[923,308,758,342,741,262,907,228]
    b=np.array(line2).reshape(4, 2)
    poly2 = Polygon(b).convex_hull

    union_poly = np.concatenate((a,b))   #合并两个box坐标，变为8*2

    if not poly1.intersects(poly2): #如果两四边形不相交
        iou = 0
    else:
        try:
            inter_area = poly1.intersection(poly2).area   #相交面积
    #         print(inter_area)
            #union_area = poly1.area + poly2.area - inter_area
            union_area = MultiPoint(union_poly).convex_hull.area
    #         print(union_area)
            if union_area == 0:
                iou= 0
            #iou = float(inter_area) / (union_area-inter_area)  #错了
            iou=float(inter_area) / union_area
    #         iou=float(inter_area) /(poly1.area+poly2.area-inter_area)
            # 源码中给出了两种IOU计算方式，第一种计算的是: 交集部分/包含两个四边形最小多边形的面积  
            # 第二种： 交集 / 并集（常见矩形框IOU计算方式） 
        except shapely.geos.TopologicalError:
            print('shapely.geos.TopologicalError occured, iou set to 0')
            iou = 0

    # print(a) 
#     print(iou)
    return iou

In [4]:
model_dir = '/remote-home/my/final/Code/OCR-code(CTPN+CRNN+CTC Loss)-baidu/checkpoints/'
model_list = os.listdir(model_dir)
model_list[0]

'v3_ctpn_ep12_0.4924_0.1245_0.6169.pth'

In [5]:
model_list.sort()
model_list[0].split("_")[2]

'ep01'

In [6]:
# iou 合适的
for model_name in model_list[3:]:
    torch.cuda.empty_cache()
    epoch = model_name.split("_")[2]
    print(epoch)
    weights = model_dir + model_name

    model = CTPN_Model()
    model.load_state_dict(torch.load(weights, map_location=device)['model_state_dict'])
    model.to(device)
    model.eval()
    
    iou_num = 0
    txt_num = 0
    pred_num = 0
    df = pd.DataFrame()
    iou_num_list = []
    txt_num_list = []
    pred_num_list = []

    val_img_dir = "/remote-home/my/final/train/img-val/"
    val_gt_dir = "/remote-home/my/final/train/gt-val/"

    # df_record_num = pd.DataFrame(columns = ['iou_num','txt_num','pred_num'])

    for idx in range(len(os.listdir(val_img_dir))):
        jpg = os.listdir(val_img_dir)[idx]
        txtname = jpg.replace("jpg","txt")

        torch.cuda.empty_cache()

    #     print(val_gt_dir + txtname)
        txt = pd.read_csv(val_gt_dir + txtname,header=None,error_bad_lines=False)
        txt_num += len(txt)
        image = cv2.imread(val_img_dir+jpg)

        try:
            text,image_after = get_det_boxes(model,image)
        except:

            print("cuda error")
            torch.cuda.empty_cache()
            continue
        pred_num += len(text)

        dfnew = pd.DataFrame(text)
        dfnew["origin_jpg"] = str(jpg).zfill(5)+".jpg"
        dfnew["index"] = dfnew.index

        df = pd.concat([df, dfnew], axis=0)
        for j in range(len(text)):
            torch.cuda.empty_cache()
            for i in range(len(txt)):
                torch.cuda.empty_cache()
                gt = list(txt.iloc[i][0:8])
                iou = cal_iou(text[j][0:8],gt)
                if iou > 0.5:
    #                 print(j)
                    iou_num += 1
        iou_num_list.append(iou_num)
        txt_num_list.append(txt_num)
        pred_num_list.append(pred_num)
        print(iou_num,txt_num,pred_num)
    df_record_num = pd.DataFrame()
    df_record_num["iou_num"]=iou_num_list
    df_record_num["txt_num"]=txt_num_list
    df_record_num["pred_num"]=pred_num_list
    df_record_num.to_csv(epoch+"_df_record_num.txt")

ep04
1 3 2
1 5 2
2 7 3
4 17 7


b'Skipping line 17: expected 10 fields, saw 11\nSkipping line 37: expected 10 fields, saw 11\n'


4 105 25
21 130 47
22 135 50
22 148 52
24 151 54
24 210 57
24 212 57
30 219 63
30 221 65
34 235 72
34 236 72
35 243 75
36 248 77
36 253 79
41 267 87
41 269 89
42 272 91
42 281 95
42 285 95
42 289 95
43 290 96


b'Skipping line 3: expected 10 fields, saw 11\nSkipping line 5: expected 10 fields, saw 11\n'


44 293 98
44 295 98
45 351 101
45 353 102
46 360 105


...

1393 7092 3270
1393 7095 3273
1394 7106 3277
1396 7115 3281
1399 7133 3306
1400 7142 3308
1401 7204 3311
1401 7206 3312
1412 7362 3339
1413 7367 3341
1413 7368 3342
1415 7370 3349
1416 7373 3353
1416 7384 3353
1418 7392 3362
1422 7400 3368
1423 7403 3371
1426 7406 3374
1430 7423 3382
1430 7424 3382
1430 7431 3382
1433 7437 3386
1433 7445 3388
1434 7446 3389
1435 7455 3393
1440 7494 3403
1441 7504 3406
1441 7513 3407
1442 7516 3409
1443 7531 3410
1443 7541 3413
1446 7544 3417
1446 7555 3419
1446 7557 3419
1448 7559 3421
1448 7564 3423
1449 7566 3424
1451 7569 3428
1453 7575 3430
1454 7581 3434
1454 7585 3434
1457 7595 3438
1459 7598 3440
1460 7601 3441
1461 7604 3445
1464 7611 3449
1465 7612 3450
1466 7626 3459
1466 7632 3461
1467 7636 3463
1469 7642 3469
1469 7643 3470
1469 7647 3472
1469 7658 3474
1481 7718 3499
1492 7732 3523
1494 7734 3527
1494 7744 3528
1494 7750 3529
1494 7751 3530
1497 7758 3539


b'Skipping line 22: expected 10 fields, saw 11\nSkipping line 25: expected 10 fields, saw 11\n'


1505 7783 3563
1507 7787 3565
1509 7792 3568
1511 7800 3571
1511 7802 3575
1513 7806 3578
1528 7836 3599
1532 7841 3605
1534 7848 3608
1534 7851 3610
1537 7876 3618
1540 7880 3622
1540 7883 3629
1541 7885 3630
1544 7889 3633
1544 7896 3635
1546 7899 3637
1546 7904 3638


b'Skipping line 27: expected 10 fields, saw 11\nSkipping line 28: expected 10 fields, saw 11\n'


1554 7940 3655
1554 7944 3657
1555 7956 3659
1556 7975 3665
1559 7988 3671
1560 7990 3677
1561 7993 3678
1563 8022 3685
1563 8026 3688
1565 8038 3691
1566 8040 3694
1570 8045 3699
1574 8049 3703
1575 8084 3710
1575 8092 3710
1575 8099 3712
1578 8106 3718
1581 8110 3721
1590 8131 3734
1592 8134 3737
1593 8141 3741
1601 8188 3757
1603 8194 3760
1603 8195 3760
1603 8200 3765
1606 8212 3770
1606 8214 3774
1607 8230 3782
1608 8239 3785
1617 8251 3798
1619 8254 3801
1620 8255 3802
1622 8281 3809
1623 8285 3811
1625 8288 3816
1630 8294 3821
1633 8300 3825
1637 8306 3832
1639 8330 3836
1646 8348 3846
1647 8351 3848
1648 8356 3850


In [12]:
df = pd.DataFrame()
for model_name in model_list[3:]:
    epoch = model_name.split("_")[2]
    num = int(epoch.replace("ep",""))
    df_new = pd.read_csv(epoch+"_df_record_num.txt")
    df_new["ep"] = num
    df = pd.concat([df, df_new], axis=0)

In [17]:
df.columns = ['pic_index', 'iou_num', 'txt_num', 'pred_num', 'ep']
df.to_csv("total_df_record_num.txt")

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

In [ ]:
!fuser -v /dev/nvidia*   

In [ ]:
!kill 5504

In [ ]:
!cat /usr/local/cuda/version.txt

In [ ]:
np.set_printoptions(suppress=True)

np.around(text, decimals=2)